# São Paulo map project ver 0.1.1
_by Gabriel P. Oliveira | 2020.08_

Including Population, Population density, Metro stations and PLWDMS (People living at walking distance from metro stations)

## 1. Purpose
The main purpose of this project is to provide a good visualization of demographic data of São Paulo city distributed over its districts. Additionally the distribution of Metro stations by districts is also provided along with an estimation of the population living at walking distance from Metro stations (here called PLWDMS).

In [ ]:
## estruturar texto
## premissas
## fontes

## 4. Analysis

In [ ]:
#clear all variables
%reset -f

In [ ]:
##might be necessary for proper installation of packages below
!conda update --all --yes

### 4.1 Installation and import of required packages

In [ ]:
import numpy as np
import pandas as pd
import timeit

import sys
!{sys.executable} -m pip install pygeos
import pygeos

#!conda install -c conda-forge libspatialindex=1.9.3 --yes
#!conda install -c conda-forge rtree=0.9.4 --yes
#import rtree

!{sys.executable} -m pip install geopandas
import geopandas as gpd

#!{sys.prefix} -m pip install folium
!conda install -c conda-forge folium=0.11.0 --yes
import folium
from folium import plugins
from folium.features import CustomIcon

import pyproj
import fiona
import fiona.crs

from shapely.geometry import Polygon, Point
from shapely.ops import unary_union, cascaded_union, transform

import branca
from branca.element import MacroElement
from jinja2 import Template

from functools import partial

# use the inline backend to generate the plots within the browser
%matplotlib inline 

import matplotlib as mpl
import matplotlib.pyplot as plt

import math

import os

### 4.2 Collecting required data

Read GeoJson file (taken from http://datageo.ambiente.sp.gov.br/geoportal/catalog/search/resource/details.page?uuid=%7B10787319-DEDC-42F7-BB0A-36CA918C4B82%7D and converted on https://ogre.adc4gis.com/) and saving it into a Geopandas dataframe

In [ ]:
sp_gpd = gpd.read_file("../MyProjects/distritos_sp_v2.json")
sp_gpd.sort_values(by=['Nome'],inplace=True)
sp_gpd.reset_index(inplace=True)
sp_gpd.drop(columns=['index', 'Codigo'], inplace=True)
sp_gpd.head()

Read population data from São Paulo in CSV and saved it to Dataframe (taken fromhttps://www.prefeitura.sp.gov.br/cidade/secretarias/urbanismo/dados_estatisticos/info_cidade/demografia/index.php?p=260265) and sorting it by population 

In [ ]:
sp_pop = pd.read_csv('../MyProjects/sp_population.csv',';',header=None,index_col=False)
sp_pop.rename(columns={0:'District', 1:'Population',2:'Density'}, inplace=True)
sp_pop.sort_values(by=['District'],inplace=True)
sp_pop.head()

Reading CSV of Metro stations in SP, renaming columns and droping duplicates (same stations in different Metro lines)

In [ ]:
sp_metro_stat = pd.read_csv('../MyProjects/sp_metro_stat2.csv',';',header=None,index_col=False)
sp_metro_stat.rename(columns={0:'Alias', 1:'Station name',2:'Line',3:'Longitude',4:'Latitude'}, inplace=True)
sp_metro_stat.drop_duplicates(subset='Station name',inplace=True,ignore_index=True)
sp_metro_stat.head()

### 4.3 Treating data

Merging population data with Geopandas data frame

In [ ]:
sp_cpl=sp_gpd.merge(sp_pop, left_on='Nome', right_on='District', how='left')

Adding string columns for latter Tooltips in the map

In [ ]:
sp_cpl['Pop str']=list(map(str, sp_cpl['Population']))
sp_cpl['Dens str']=list(map(str, sp_cpl['Density']))
sp_cpl.head()

Locating Metro stations into the districts and counting

In [ ]:
#creating points geometry from Latitude and Longitude data
sp_metro_stat_gpd=gpd.GeoDataFrame(sp_metro_stat,geometry=gpd.points_from_xy(sp_metro_stat.Longitude,
                                                                             sp_metro_stat.Latitude))
#locating both geometries (districts and stations) into the same projection CRS
sp_cpl.crs=fiona.crs.from_epsg(4326)
sp_metro_stat_gpd.crs=fiona.crs.from_epsg(4326)
#locating 
sjoined_sp_metro_stat = gpd.sjoin(sp_metro_stat_gpd, sp_cpl, op='within')
#counting
sp_metro_stat_grouped = sjoined_sp_metro_stat.groupby('Nome').size()
sp_mstat_grouped_df = sp_metro_stat_grouped.to_frame().reset_index()
sp_mstat_grouped_df.columns = ['Distrito','Metro stations']
sp_mstat_grouped_df.head()

Merging Metro stations number into the complete dataframe and adding a string column for Tooltip on map

In [ ]:
sp_cpl=sp_cpl.merge(sp_mstat_grouped_df, left_on='Nome', right_on='Distrito', how='left')
sp_cpl.drop(columns=['District','Distrito'],inplace=True)
sp_cpl['Metro stations'].fillna(0,inplace=True)
sp_cpl['Metro stations']=sp_cpl['Metro stations'].astype('int64')
sp_cpl['M stat str']=list(map(str, sp_cpl['Metro stations']))

Calculating the total Population and total number of Metro stations

In [ ]:
total_pop=sp_cpl['Population'].sum()
total_metro_stat=sp_cpl['Metro stations'].sum()
print('The total population is',total_pop,'ppl.')
print('The total number of metro stations is',total_metro_stat,'stations')

#### 4.3.1 Estimation of number of people living at walking distance (PLWDMS) from metro stations

Calculating the areas from the Districts geometries (GeoJson file) and the areas from the official data (District Population divided by the Pop. Density)

In [ ]:
areas_geojson=sp_gpd['geometry'].to_crs('EPSG:32723').map(lambda p: p.area / 1000000)
#EPSG 31983 and 29193 were also tested and provided the same result
areas_database=sp_cpl['Population'].div(sp_cpl['Density'],axis=0)

This website was used as reference to choose the Projections http://processamentodigital.com.br/2013/07/27/lista-dos-codigos-epsg-mais-utilizados-no-brasil/

Checking the difference in the area calculated by geometry and from official data

In [ ]:
diff=areas_geojson/areas_database
diff=(diff-1)*100
diff_pd=diff.to_frame()
diff_pd=diff_pd.round(2)
diff_pd.rename(columns={0:'Area diff'}, inplace=True)
diff_pd=diff_pd.join(sp_cpl[['geometry','Nome']])
diff_pd['A_diff str']=list(map(str, diff_pd['Area diff']))
diff_gpd=gpd.GeoDataFrame(diff_pd, geometry='geometry')
diff_gpd.crs=fiona.crs.from_epsg(4326)
diff_gpd.head()

In [ ]:
mpl.style.use('ggplot') # optional for changing style
mybins=list(map(int,np.linspace(-16,16,num=17,endpoint=True)))
plt.figure(figsize=(14,7))
plt.hist(diff_pd['Area diff'], bins=mybins,rwidth=0.9)
plt.xticks(mybins,mybins)
plt.title('Difference in area calculation from GeoJson file and official data')
plt.ylabel('# of Districts')
plt.xlabel('% Difference in area')
plt.show()

Due to this difference, it was decided to not use the official density for further calculations. Further discussion and investigation on the area difference is carried out on section 4.6.

Therefore the calculation of the number of people living at walking distance from metro stations is carried out by getting the "walking distance" area around a metro station in a District divided by the District's area and multipling by its Population. The walking distance area here is considered a 1 km radius around the metro station.

In [ ]:
#function to create a circle area around a point
proj_wgs84 = pyproj.Proj('+proj=longlat +datum=WGS84')

def geodesic_point_buffer(lat, lon, km):
    # Azimuthal equidistant projection
    aeqd_proj = '+proj=aeqd +lat_0={lat} +lon_0={lon} +x_0=0 +y_0=0'
    project = partial(
        pyproj.transform,
        pyproj.Proj(aeqd_proj.format(lat=lat, lon=lon)),
        proj_wgs84)
    buf = Point(0, 0).buffer(km * 1000)  # distance in metres
    return transform(project, buf).exterior.coords[:]

In [ ]:
walk_dist=1 #walking distance in km
sp_metro_stat_poly=sp_metro_stat_gpd.copy()
sp_metro_stat_poly.drop(columns=['Longitude','Latitude','geometry'],inplace=True)
sp_metro_stat_poly['geometry']=Polygon()
for k in range(sp_metro_stat_gpd.shape[0]):
    b=geodesic_point_buffer(sp_metro_stat_gpd.loc[k,'Latitude'],sp_metro_stat_gpd.loc[k,'Longitude'],walk_dist)
    sp_metro_stat_poly.loc[k,'geometry']=Polygon(b)
sp_metro_stat_poly.head()

Checking if the areas around the metro stations are consistent (pi*radius²)

In [ ]:
areas_stations=sp_metro_stat_poly['geometry'].to_crs('EPSG:32723').map(lambda p: p.area / 1000000)
areas_stations.describe()

Joining all geometry areas around metro stations and intersecting the total area geometry with each district geometry

In [ ]:
geometries = gpd.GeoSeries(sp_metro_stat_poly['geometry'])
total_poly=geometries.unary_union
intersec_metro=sp_cpl['geometry'].intersection(total_poly)
intersec_metro=gpd.GeoDataFrame(intersec_metro)
intersec_metro.rename(columns={0:'geometry'}, inplace=True)
intersec_metro.head()

Calculating the instersection areas and calculating the estimation of PLWDMS

In [ ]:
areas_plwdms=intersec_metro['geometry'].to_crs('EPSG:32723').map(lambda p: p.area / 1000000)
plwdms=sp_cpl['Population']*areas_plwdms/areas_geojson
plwdms=round(plwdms,0)
plwdms=list(map(int,plwdms))
sp_cpl['PLWDMS']=plwdms
sp_cpl['PLWDMS str']=list(map(str, sp_cpl['PLWDMS']))

Calculating the total PLWDMS and city Pop. Density

In [ ]:
total_plwdms=sp_cpl['PLWDMS'].sum()
total_area_db=areas_database.sum()
total_area_geo=areas_geojson.sum()
total_dens_db=total_pop/total_area_db
total_dens_geo=total_pop/total_area_geo
print('The estimated total number of people living at walking distance from metro stations is',total_plwdms,'ppl.')
print('The population density of the city (based on database area) is',int(total_dens_db),'ppl./km^2')
print('The population density of the city (based on geojson area) is',int(total_dens_geo),'ppl./km^2')

### 4.4 Creating the map layers and setting its properties 

Creating colorscales for the different layers on the map

In [ ]:
# definition of the max value on the scale
# using the first algarism and rounding it up ## for latter versions: create a function "def"

ten_log=list(map(int,np.logspace(0,9,10)))

ordem=0
for i in range(10):
    if sp_cpl['Population'].max()<ten_log[i]:
        ordem=ten_log[i-1]
        break
first_alg=sp_cpl['Population'].max()//ordem+(sp_cpl['Population'].max()%ordem>0) #rounding up
max_pop_scale=first_alg*ordem

for i in range(10):
    if sp_cpl['Density'].max()<ten_log[i]:
        ordem=ten_log[i-1]
        break
first_alg=sp_cpl['Density'].max()//ordem+(sp_cpl['Density'].max()%ordem>0) #rounding up
max_dens_scale=first_alg*ordem

for i in range(10):
    if sp_cpl['PLWDMS'].max()<ten_log[i]:
        ordem=ten_log[i-1]
        break
first_alg=sp_cpl['PLWDMS'].max()//ordem+(sp_cpl['PLWDMS'].max()%ordem>0) #rounding up
max_plwdms_scale=first_alg*ordem

#creating colorscales
colorscale_pop=folium.LinearColormap(['white','red'],vmin=0,vmax=max_pop_scale)
colorscale_pop.caption='Population [ppl]'
colorscale_dens=folium.LinearColormap(['white','purple'],vmin=0,vmax=max_dens_scale)
colorscale_dens.caption='Pop. Density [ppl/km^2]'
colorscale_mstat=folium.LinearColormap(['white','green'],vmin=0,vmax=sp_cpl['Metro stations'].max())
colorscale_mstat.caption='Metro stations'
colorscale_plwdms=folium.LinearColormap(['white','orange','#F85B14'],index=[0,max_plwdms_scale/2,max_plwdms_scale],vmin=0,vmax=max_plwdms_scale)
colorscale_plwdms.caption='PLWDMS [ppl]'

Creating the map layers

In [ ]:
sp_map_lay=folium.Map(location=[-23.68, -46.45], zoom_start=10, tiles='CartoDB Positron', width=800, height=500)

#layer for population
sp_pop_map=folium.GeoJson(sp_cpl[['geometry','Nome','Population','Pop str']],
               name="São Paulo Population",
               style_function=lambda x: {"weight":1, 'color':'white','fillColor':colorscale_pop(x['properties']['Population']), 'fillOpacity':0.7},
              highlight_function=lambda x: {'weight':2, 'color':'red'},
               #smooth_factor=2.0,
              tooltip=folium.features.GeoJsonTooltip(fields=['Nome','Pop str'],
                                              aliases=['District','Population'], 
                                              labels=True, 
                                              sticky=True,
                                              toLocaleString=True
                                             )
                         )
sp_pop_map.add_to(sp_map_lay)
colorscale_pop.add_to(sp_map_lay)

#Layer for population density
sp_dens_map=folium.GeoJson(sp_cpl[['geometry','Nome','Density','Dens str']],
               name="São Paulo Pop. Density",
               style_function=lambda x: {"weight":1, 'color':'white','fillColor':colorscale_dens(x['properties']['Density']), 'fillOpacity':0.7},
              highlight_function=lambda x: {'weight':2, 'color':'purple'},
               #smooth_factor=2.0,
               show=False,
              tooltip=folium.features.GeoJsonTooltip(fields=['Nome','Dens str'],
                                              aliases=['District','Pop. Density'], 
                                              labels=True, 
                                              sticky=True,
                                              toLocaleString=True
                                             )
              )
sp_dens_map.add_to(sp_map_lay)
colorscale_dens.add_to(sp_map_lay)

#Layer for Metro stations
sp_mstat_map=folium.GeoJson(sp_cpl[['geometry','Nome','Metro stations','M stat str']],
               name="Metro Stations #",
               style_function=lambda x: {"weight":1, 'color':'white','fillColor':colorscale_mstat(x['properties']['Metro stations']), 'fillOpacity':0.7},
              highlight_function=lambda x: {'weight':2, 'color':'green'},
               #smooth_factor=2.0,
               show=False,
              tooltip=folium.features.GeoJsonTooltip(fields=['Nome','M stat str'],
                                              aliases=['District','# of Metro stations'], 
                                              labels=True, 
                                              sticky=True,
                                              toLocaleString=True
                                             )
              )
sp_mstat_map.add_to(sp_map_lay)
colorscale_mstat.add_to(sp_map_lay)

#Layer for PLWDMS
sp_plwdms_map=folium.GeoJson(sp_cpl[['geometry','Nome','PLWDMS','PLWDMS str']],
               name="PLWDMS",
               style_function=lambda x: {"weight":1, 'color':'white','fillColor':colorscale_plwdms(x['properties']['PLWDMS']), 'fillOpacity':0.7},
              highlight_function=lambda x: {'weight':2, 'color':'orange'},
               #smooth_factor=2.0,
               show=False,
              tooltip=folium.features.GeoJsonTooltip(fields=['Nome','PLWDMS str'],
                                              aliases=['District','PLWDMS'], 
                                              labels=True, 
                                              sticky=True,
                                              toLocaleString=True
                                             )
              )
sp_plwdms_map.add_to(sp_map_lay)
colorscale_plwdms.add_to(sp_map_lay)

Creating Metro Icon and Adding Markers to the stations

In [ ]:
metro_png = os.path.abspath('../MyProjects/São_Paulo_Metro_logo.png')
metro_marker=folium.FeatureGroup(name='Metro Station markers',show=False)

for h in range(sp_metro_stat.shape[0]):
    metro_icon = folium.CustomIcon(metro_png,icon_size=(15, 15)) #the custom icon has to reset each iteration (folium bug)
    a=folium.Marker(
        location=[sp_metro_stat.loc[h,'Latitude'],sp_metro_stat.loc[h,'Longitude']],
        popup=sp_metro_stat.loc[h,'Station name'],
        icon=metro_icon
        )
    metro_marker.add_child(a)

metro_marker.add_to(sp_map_lay)

Adding layer control, fullscreen and a secondary base layer

In [ ]:
folium.TileLayer('openstreetmap').add_to(sp_map_lay)
folium.plugins.Fullscreen(position='bottomright', title='Full Screen', title_cancel='Exit Full Screen', force_separate_button=False).add_to(sp_map_lay)
folium.LayerControl('topright',autoZIndex=False, collapsed=True).add_to(sp_map_lay)

Combining colorscales and layers for the controls

In [ ]:
#creating a function to combine
class BindColormap(MacroElement):
    """Binds a colormap to a given layer.

    Parameters
    ----------
    colormap : branca.colormap.ColorMap
        The colormap to bind.
    """
    def __init__(self, layer, colormap):
        super(BindColormap, self).__init__()
        self.layer = layer
        self.colormap = colormap
        self._template = Template(u"""
        {% macro script(this, kwargs) %}
            {{this.colormap.get_name()}}.svg[0][0].style.display = 'block';
            {{this._parent.get_name()}}.on('overlayadd', function (eventLayer) {
                if (eventLayer.layer == {{this.layer.get_name()}}) {
                    {{this.colormap.get_name()}}.svg[0][0].style.display = 'block';
                }});
            {{this._parent.get_name()}}.on('overlayremove', function (eventLayer) {
                if (eventLayer.layer == {{this.layer.get_name()}}) {
                    {{this.colormap.get_name()}}.svg[0][0].style.display = 'none';
                }});
        {% endmacro %}
        """)

#combining
sp_map_lay.add_child(BindColormap(sp_mstat_map,colorscale_mstat))
sp_map_lay.add_child(BindColormap(sp_pop_map,colorscale_pop))
sp_map_lay.add_child(BindColormap(sp_dens_map,colorscale_dens))
sp_map_lay.add_child(BindColormap(sp_plwdms_map,colorscale_plwdms))
print('Layers and colorscales combined')

### 4.5 Display map

In [ ]:
sp_map_lay

Save the map in html format if wanted

In [ ]:
file_name='SP_map_0.1.1.html'
sp_map_lay.save(file_name)

## 4.6 Discussion on area difference

In order to investigate the area difference between the GeoJson file and the database used for population and density, the % area difference was plot in a map.

In [ ]:
colorscale_adiff=folium.LinearColormap(['blue','white','red'],vmin=-16,vmax=16)
colorscale_adiff.caption='Area difference [%]'

In [ ]:
#Map for Area difference
sp_map_diff=folium.Map(location=[-23.68, -46.45], zoom_start=10, tiles='CartoDB Positron', width=800, height=500)
sp_adiff_map=folium.GeoJson(diff_gpd[['geometry','Nome','Area diff','A_diff str']],
               name="Area difference",
               style_function=lambda x: {"weight":1, 'color':'white','fillColor':colorscale_adiff(x['properties']['Area diff']), 'fillOpacity':0.7},
              highlight_function=lambda x: {'weight':2, 'color':'black'},
               #smooth_factor=2.0,
               show=False,
              tooltip=folium.features.GeoJsonTooltip(fields=['Nome','A_diff str'],
                                              aliases=['District','Area difference'], 
                                              labels=True, 
                                              sticky=True,
                                              toLocaleString=True
                                             )
              )
sp_adiff_map.add_to(sp_map_diff)
colorscale_adiff.add_to(sp_map_diff)
sp_map_diff

There is no clear reason for the area difference by observing the map. Therefore other possibilities were check.

Checking if there is any relationship between the % area difference and the district area.

In [ ]:
area_pd=pd.DataFrame(areas_geojson)
area_pd['Nome'] = sp_cpl['Nome']
area_pd['Area diff'] = diff_pd['Area diff']
area_pd.rename(columns={'geometry':'Area GJSON'}, inplace=True)
area_pd['Area data'] = areas_database
area_pd.head()

In [ ]:
area_pd.plot(kind='scatter', x='Area GJSON', y='Area diff', figsize=(14, 7), color='darkblue')

plt.title('Relationship between GEOJson area and Area difference')
plt.xlabel('GeoJson Area')
plt.ylabel('% Area difference')

plt.show()

There is no clear relationship between the % area difference and the district area.

Another possibility that can be tested is if Districts with complex geometries influences the area difference. To determine the District geometry complexity, the shape index is used. The shape index is a ratio between the perimeter of the District and the perimeter of a regular geometry (in this case a square) of the same area. Higher shape index means a distorted geometry compared to the regular one.

In [ ]:
perimeters=sp_cpl['geometry'].to_crs('EPSG:32723').map(lambda p: p.length / 1000)
sqr_per=areas_geojson.map(lambda p: math.sqrt(p) * 4)
shape_index=perimeters/sqr_per

In [ ]:
shape_index = pd.DataFrame(shape_index)
shape_index['Nome'] = sp_cpl['Nome']
shape_index['Area diff'] = diff_pd['Area diff']
shape_index.rename(columns={'geometry':'Shape index'}, inplace=True)
shape_index.head()

In [ ]:
shape_index.plot(kind='scatter', x='Shape index', y='Area diff', figsize=(14, 7), color='darkblue')

plt.title('Relationship between District shape index and Area difference')
plt.xlabel('Shape index')
plt.ylabel('% Area difference')

plt.show()

There is no clear relationship between the shape index and the area difference. But just for curiosity let's see the shape index on a map.

In [ ]:
shape_index['geometry']=sp_cpl['geometry']
shape_index['Shp_idx str']=list(map(str, shape_index['Shape index']))
shp_idx_gpd=gpd.GeoDataFrame(shape_index, geometry='geometry')
shp_idx_gpd.crs=fiona.crs.from_epsg(4326)
shp_idx_gpd.head()

In [ ]:
#Map for Shape index
sp_map_shpidx=folium.Map(location=[-23.68, -46.45], zoom_start=10, tiles='CartoDB Positron', width=800, height=500)
colorscale_shpidx=folium.LinearColormap(['blue','white','red'],index=[0.9,1,1.9],vmin=0.9,vmax=1.9)
colorscale_shpidx.caption='Shape index'
sp_shpidx_lay=folium.GeoJson(shp_idx_gpd[['geometry','Nome','Shape index','Shp_idx str']],
               name="Shape index",
               style_function=lambda x: {"weight":1, 'color':'white','fillColor':colorscale_shpidx(x['properties']['Shape index']), 'fillOpacity':0.7},
              highlight_function=lambda x: {'weight':2, 'color':'black'},
               #smooth_factor=2.0,
               show=True,
              tooltip=folium.features.GeoJsonTooltip(fields=['Nome','Shp_idx str'],
                                              aliases=['District','Shape index'], 
                                              labels=True, 
                                              sticky=True,
                                              toLocaleString=True
                                             )
              )
sp_shpidx_lay.add_to(sp_map_shpidx)
colorscale_shpidx.add_to(sp_map_shpidx)
sp_map_shpidx

It is interesting to see that central Districits have lower shape indexes (more regular geometries) that the ones at the borders, porbably because the borders are usually defined by natural lines (e.g. rivers).
Santo Amaro district has a shape index lower than 1. That can be explained by its regular geometry which is almost a regular square with a rounded corner (a circle has a lower shape index compared to a square).

Another possibility to explain the area difference is a possible overlap between the districts. To check this the areas of each district will be summed and then a new geometry will be created by joining all districs geometries and the area of this total geometry will be calculated. If there is a difference between this two calculations, this means that some distritcs geometries are overlapped.

In [ ]:
areas_geojson.sum()

In [ ]:
districts_geom=gpd.GeoSeries(sp_cpl['geometry'])
sp_geom=districts_geom.unary_union
sp_geom

In [ ]:
sp_geom=gpd.GeoSeries(sp_geom)
sp_geom.crs=fiona.crs.from_epsg(4326)
total_area=sp_geom.to_crs('EPSG:32723').map(lambda p: p.area / 1000000)
total_area

There is no difference in area, therefore there is no significant overlap between district geometries.

Now, let's check the total area of São Paulo using the population and density database values and compare it with the geojson files.

In [ ]:
areas_database.sum()

In [ ]:
total_diff=100*(areas_geojson.sum()/areas_database.sum()-1)

In [ ]:
print('The total area difference is',round(total_diff,2),'%')

Let's try to check a different source for the area. According to IBGE, the area of São Paulo city is 1521 km². The GeoJson file from IBGE source was then loaded and the area from this file was calculated using the same method used for the districts.

In [ ]:
sp_cities = gpd.read_file("../MyProjects/sp_cities.json")
sp_cities.head()

In [ ]:
sp_cities.loc[lambda z: z['NM_MUN'] == 'São Paulo']

In [ ]:
sp_cities.crs=fiona.crs.from_epsg(4326)
sp_cities_areas=sp_cities['geometry'].to_crs('EPSG:32723').map(lambda p: p.area / 1000000)
total_area_ibge=sp_cities_areas.iloc[562]

In [ ]:
total_area_ibge

The calculated area is the same as the official data from IBGE. This supports the usage of the area from the geojson file for the districts instead of the database used for population and density. The difference in area is probably due to a different measurement technique or different borders considered for the districts.
Therefore the usage of the GeoJson file for the area calculation seems to be a beter choice for the present analysis.

## 6.0 Conclusions and Comments

The maps provided a good visualization of demographic data of São Paulo city distributed over its districts together with the distribution of Metro stations and the estimation of the population living at walking distance from Metro stations (here called PLWDMS).

Data has shown that Grajaú is the district with the higher population (360787 people) while Marsilac is the one with the lowest (8258 people). Marsilac also has the largest area (208.2 km²) and therefore the lowest population density (41 ppl/km²). Sé is the smallest district (2.2 km²) and Bela vista is the district with higher population density (26715 ppl/km²)

In [ ]:
print(sp_cpl.loc[sp_cpl['Population'].idxmax(),'Nome'],'population is',sp_cpl['Population'].max())
print(sp_cpl.loc[sp_cpl['Population'].idxmin(),'Nome'],'population is',sp_cpl['Population'].min())
print(area_pd.loc[area_pd['Area GJSON'].idxmax(),'Nome'],'area is',round(area_pd['Area GJSON'].max(),1),'km^2')
print(area_pd.loc[area_pd['Area GJSON'].idxmin(),'Nome'],'area is',round(area_pd['Area GJSON'].min(),1),'km^2')
print(sp_cpl.loc[sp_cpl['Density'].idxmax(),'Nome'],'pop. density is',sp_cpl['Density'].max(),'ppl/km^2')
print(sp_cpl.loc[sp_cpl['Density'].idxmin(),'Nome'],'pop. density is',sp_cpl['Density'].min(),'ppl/km^2')

Two districts (Santo Amaro and Vila Mariana) have the higest number of Metro stations inside its boundaries. While 59 districts have none (which is 61.5% of the total number of districts).

In [ ]:
sp_cpl.loc[sp_cpl['Metro stations']==sp_cpl['Metro stations'].max(),['Nome','Metro stations']]

In [ ]:
sp_cpl.loc[sp_cpl['Metro stations']==sp_cpl['Metro stations'].min(),['Nome','Metro stations']]

In [ ]:
z=sp_cpl.loc[sp_cpl['Metro stations']==sp_cpl['Metro stations'].min(),'Nome'].count()
print('Number of districts without a Metro station: ',z)
print('% of total districts: ',100*round(z/sp_cpl.shape[0],3),'%')

In [ ]:
mybins=list(map(int,np.linspace(0,6,num=7,endpoint=True)))
plt.figure(figsize=(8,4))
plt.hist(sp_cpl['Metro stations'], bins=mybins,rwidth=0.9)
plt.xticks(mybins,mybins)
plt.title('Distribution of districts and their number of metro stations')
plt.ylabel('# of Districts')
plt.xlabel('# of Metro stations')
plt.show()

It is insteresting to see in the map that Campo Belo and Cambuci districts are surrounded by districts with metro stations but have none within their own boundaries.

The estimation of total PLWDMS over 2 million people (which represents 18.1% of the total population) seems to be rather over estimated. This is probably due to the method used for the calculation which considers the population density of the complete district while the area closer to the Metro stations might have considerably different population density.
Metro stations can be located in a more commercial area of the district rather than an residencial area, also Metro stations that are the Terminal station of a line may have large parking areas for trains which will occupy much of the considered area.
For a better estimation of PLWDMS, population data with higher spatial resolution than district areas would be necessary.

In [ ]:
perc_plwdms=total_plwdms/total_pop
perc_plwdms

Sapopemba is the district with higher PLWDMS (192909 ppl, which represents 9.4% of the total PLWDMS), that's because it has a high density and has 5 metro stations. The following calculation demonstrates that 2/3 of its area is covered by a 1km radius around metro stations. Lapa is the district with lowest PLWDMS (excluding the ones with 0 value) of only 52 people.

In [ ]:
print(sp_cpl.loc[sp_cpl['PLWDMS'].idxmax(),'Nome'],'PLWDMS is',sp_cpl['PLWDMS'].max(),'ppl')
print('% of total PLWDMS: ',100*round(sp_cpl['PLWDMS'].max()/total_plwdms,3),'%')
print(sp_cpl.loc[sp_cpl['PLWDMS'][sp_cpl['PLWDMS']>0].idxmin(),'Nome'],'PLWDMS is',sp_cpl['PLWDMS'][sp_cpl['PLWDMS']>0].min(),'ppl')

In [ ]:
ratio=sp_cpl['PLWDMS'].loc[77]/sp_cpl['Population'].loc[77]
ratio

It is interesting to notice that Penha district has no metro station within its boundaries but has a PLWDMS number of 59119 people (the 11th largest). That's because 4 metro stations are located right next to its borders. The list of the top 15 is given below.

In [ ]:
print(sp_cpl.loc[sp_cpl['PLWDMS'][sp_cpl['Metro stations']==0].idxmax(),'Nome'],'PLWDMS is',sp_cpl['PLWDMS'][sp_cpl['Metro stations']==0].max(),'ppl')
rank=sp_cpl[['Nome','PLWDMS']].sort_values(by='PLWDMS',ascending=False)
rank.reset_index(drop=True,inplace=True)
rank.head(15)

There are actually 17 districts that have no metro stations within its borders but have PLWDMS higher than zero. The list of all these districts is shown below.

In [ ]:
a=sp_cpl[(sp_cpl['PLWDMS']>0) & (sp_cpl['Metro stations']==0)]

In [ ]:
print('Districts with no metro stations and PLWDMS >0: ',a.shape[0])
a[['Nome','PLWDMS']].sort_values(by='PLWDMS',ascending=False)

The data used in this analysis is from the 2010 census, therefore, it can present significant difference from the actual status.

## 7.0 Next steps

Improvements to be considered for better/further analysis: <br>
-Use lattest census data (postponed to 2021 by IBGE due to the corona pandemic);<br>
-Use the IBGE census regions (and respective population density) to determine the PLWDMS as they smaller than the districts, therefore providing a better spatial resolution for population densisty;<br>
-Insert static legends on the map with the total number for each layer;<br>
-Add CPTM stations to the map.

## 8.0 References

Function to bind the colormap scale into the layer: https://nbviewer.jupyter.org/gist/BibMartin/f153aa957ddc5fadc64929abdee9ff2e <br>
Function to draw a circle polygon around a geographic point: https://stackoverflow.com/questions/62899651/is-it-possible-to-azimuthal-equidistant-projection-and-create-a-buffer-polygon-i